In [ ]:
import sel_scrape as sc
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
import requests
from requests.auth import HTTPBasicAuth
import os
import time
import traceback
from tqdm import tqdm,tqdm_notebook


In [ ]:
# import importlib
# importlib.reload(sc)

In [ ]:
ACS_HOME_PAGE = 'http://acs.barchart.com/mri/mripag.htm' 
ACS_LOGIN_INFO = './temp_folder/acs_login.txt'
acs_text = open(ACS_LOGIN_INFO,'r').read()
ACS_USERNAME = acs_text.split(',')[0]
ACS_PASSWORD = acs_text.split(',')[1]
ZIP_FOLDER_PARENT = open('./temp_folder/zip_folder_parent.txt','r').read()

In [ ]:
sela = sc.SelScrape(headless=False)
sela.goto(ACS_HOME_PAGE)
time.sleep(1)
wait(sela.driver, 5).until(EC.alert_is_present())
alert = sela.driver.switch_to_alert()
alert.send_keys(f'{ACS_USERNAME}{Keys.TAB}{ACS_PASSWORD}')
time.sleep(1)
alert.accept()


In [ ]:
eod_files_xpath = "//a[@href='mrifile.htm']"
eod_elem = sela.findxpath(eod_files_xpath)['value'][0]
sela.goto(eod_elem.get_attribute('href'))
opt_eod_xpath = "//a[@href='mriopt.htm']"
opt_eod_elem = sela.findxpath(opt_eod_xpath)['value'][0]
sela.goto(opt_eod_elem.get_attribute('href'))


In [ ]:
monthly_csv_files_xpath = "//a[contains(@href,'data/opt/opv')]"
mcsv_elements = sela.findxpath(monthly_csv_files_xpath)['value']
mcsv_hrefs = []
for mcsv in mcsv_elements:
    mcsv_hrefs.append(mcsv.get_attribute('href'))
mcsv_hrefs

In [ ]:
options_parent = ZIP_FOLDER_PARENT+'/options'
hrefs_to_unzip = []
paths_to_unzip_to = []
for mcsv_href in mcsv_hrefs:
    zip_file_name = mcsv_href.split('/')[-1]
    folder_name = zip_file_name.replace('.zip','')
    path_to_zip_folder = f'{options_parent}/{folder_name}'
    if not os.path.isdir(path_to_zip_folder):
        print(f'making {path_to_zip_folder}')
        os.mkdir(path_to_zip_folder)
    path_to_zip_file = f'{path_to_zip_folder}/{zip_file_name}'
    if not os.path.isfile(path_to_zip_file):
        hrefs_to_unzip.append(mcsv_href)
        paths_to_unzip_to.append(path_to_zip_file)

successful_downloads = []
for i in tqdm_notebook(range(len(hrefs_to_unzip))):
    url = hrefs_to_unzip[i]
    path_to_zip_file = paths_to_unzip_to[i]
    try:    
        r=requests.get(url, auth=HTTPBasicAuth(ACS_USERNAME, ACS_PASSWORD))
        p = paths_to_unzip_to[i]
        with open(p, 'wb') as f:
            f.write(r.content)
        successful_downloads.append(path_to_zip_file)
    except Exception as e:
        traceback.print_exc()
        

        